#### Data

You will need the following data files in your directory: 


- [data/OIL_AND_GAS_LOCATIONS_SHP/Oil_and_Gas_Locations.shp](https://cogcc.state.co.us/documents/data/downloads/gis/metadata/Oil_and_Gas_Locations_Metadata.html)
- [data/TANK_BATTERIES_SHP/Tank_Batteries.shp](https://cogcc.state.co.us/documents/data/downloads/gis/metadata/Tank_Batteries_Metadata.html)
- [data/PITS_SHP/Pits.shp](https://cogcc.state.co.us/documents/data/downloads/gis/metadata/Pits_Metadata.html)
- [data/tl_2019_08_tabblock10/tl_2019_08_tabblock10.shp](https://catalog.data.gov/dataset/tiger-line-shapefile-2019-2010-state-colorado-2010-census-block-state-based)
- data/wells_with_distance_meters/wells_with_distance_meters.shp ([available in repo](https://github.com/karmijo/mapping-for-environmental-justice/tree/master/data))
- data/tanks_with_distance_meters/tanks_with_distance_meters.shp ([available in repo](https://github.com/karmijo/mapping-for-environmental-justice/tree/master/data))
- data/pits_with_distance_meters/pits_with_distance_meters.shp ([available in repo](https://github.com/karmijo/mapping-for-environmental-justice/tree/master/data))
- [data/Colorado_Census_Tract_Boundaries-shp/Colorado_Census_Tract_Boundaries.shp](https://data-cdphe.opendata.arcgis.com/datasets/a9f5b1a67bd74b2fa22279d141625335_3/data)
- ['data/EJSCREEN_2019_USPR.csv](ftp://newftp.epa.gov/EJSCREEN/2019/)
- data/dieselpmexposure.csv ([available in repo](https://github.com/karmijo/mapping-for-environmental-justice/tree/master/data))
- [data/nata2014v2_national_resphi_by_tract_poll.xlsx](https://www.epa.gov/sites/production/files/2018-08/nata2014v2_national_resphi_by_tract_poll.xlsx)
- [data/2012thru2016-140-csv/2012thru2016-140-csv/140/Table8.csv](https://www.huduser.gov/portal/datasets/cp.html)
- [data/Asthma_Prevalence_in_Adults_-_CDPHE_Community_Level_Estimates__Census_Tracts_.csv](https://data-cdphe.opendata.arcgis.com/datasets/asthma-prevalence-in-adults-cdphe-community-level-estimates-census-tracts)
- [data/Heart_Disease_in_Adults_-_CDPHE_Community_Level_Estimates__Census_Tracts_.csv](https://data-cdphe.opendata.arcgis.com/datasets/heart-disease-in-adults-cdphe-community-level-estimates-census-tracts)
- [data/Low_Weight_Birth_Rate__Census_Tracts_.csv](https://data-cdphe.opendata.arcgis.com/datasets/low-weight-birth-rate-census-tracts)








#### Packages

In [2]:
import geopandas as gpd
import math
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
import pandas as pd
import pymysql
import requests
import sqlite3
import zipfile
from pandas.io import sql
from sqlalchemy import create_engine

### 2. Oil & Gas <a id='oil'></a>

The Oil & Gas indicator represents oil and gas facilities that include wells, wastewater pits, and tanks of oil or other dangerous chemicals located near populated areas. Oil and gas operations have been linked to many health problems in nearby communities including headaches, nausea, cancer, asthma, and birth defects.

The Oil & Gas indicator measures the count in each census tract of all oil and gas facilities, including wells, tanks, pits, within 1 kilometer of a populated census block, where each site is weighted by it’s distance to the nearest populated census block. Oil wells, tanks, and pits location data from 1999-2020 is sourced from the [Colorado Oil and Gas Conservation Commission](https://cogcc.state.co.us/data2.html#/downloads). The distancing weighting method is modelled after CalEnviroScreen’s method for distance weighting using the same following weights: 1 if the site is less than 250 meters from a populated census block, 0.5 if the site is 250-500 meters from a populated census block, 0.25 if the site is 500-750 meters from a populated census block, and 0.1 if the site is 750-1000 meters from a populated census block. All facilities further than 1,000 meters from any populated census blocks were excluded (CalEnviroScreen, 2017).

- Data for the number of wells can be found [here](https://cogcc.state.co.us/documents/data/downloads/gis/metadata/Oil_and_Gas_Locations_Metadata.html) under "Oil & Gas Locations (3.7 Mb)".
- Data for the number of pits can be found [here](https://cogcc.state.co.us/documents/data/downloads/gis/metadata/Pits_Metadata.html) under "Pits (1 Mb)".
- Data for the number of tank batteries can be found [here](https://cogcc.state.co.us/documents/data/downloads/gis/metadata/Tank_Batteries_Metadata.html) under "Tank Batteries (87 Kb)".

Note: The [state](https://cogcc.state.co.us/documents/about/COGIS_Help/Status_Codes.pdf) of the well (ex. not active anymore, just being drilled, etc.), where the older the well is the less harmful it is currently, is not currently incorporated in this analysis.

Finally, this indicator is only calculated for the state of Colorado.

In [3]:
# Oil and gas facilities data: https://cogcc.state.co.us/documents/data/downloads/gis/metadata/Oil_and_Gas_Locations_Metadata.html
oil_gas = gpd.read_file("data/OIL_AND_GAS_LOCATIONS_SHP/Oil_and_Gas_Locations.shp")

In [5]:
# Tank battery data: https://cogcc.state.co.us/documents/data/downloads/gis/metadata/Tank_Batteries_Metadata.html
# Tank Battery is a device used to store crude oil which is produced from a well.
tank_batteries = gpd.read_file("data/TANK_BATTERIES_SHP/Tank_Batteries.shp")

In [6]:
# Oil pits data: https://cogcc.state.co.us/documents/data/downloads/gis/metadata/Pits_Metadata.html
pits = gpd.read_file("data/PITS_SHP/Pits.shp")

In [7]:
# Create empty list for non-populated blocks, which will be filtered by GEOID to get unique codes
nonpop_blocks = []

# Get all county codes in Colorado: https://simple.wikipedia.org/wiki/List_of_counties_in_Colorado
# 001 - 125 by odd nums
counties = list(map(str, np.arange(1, 127, 2)))
counties = list(map(lambda x: str.zfill(x, 3), counties))

# Get population data using the Census Decennial Summary File 1 (SF1) from 2010 because it goes down to block level
# ACS5 has more recent population data from 2018, but only goes down to block group level
# Find Decennial SF1 in 2010 at https://api.census.gov/data.html 
# Colorado is state code 08
# Must use for loop over all counties because api doesn't allow iteration over all blocks at once 
for county_code in counties: 
    url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,group(P1)&for=block:*&in=state:08%county:" + county_code
    r = requests.get(url)

    r.raise_for_status()
    
    data = r.json()

    blocks = pd.DataFrame(data)
    blocks.columns = blocks.iloc[0]
    blocks = blocks.iloc[1:]

    # Convert total population data to integers
    # P001001	Total	TOTAL POPULATION
    blocks['P001001'] = blocks['P001001'].apply(int)
    
    # Add block geoids where total population is 0 to the nonpopulated list
    nonpop_blocks.extend(blocks[blocks['P001001'] == 0]['GEO_ID'].values.tolist())

# Get rid of first '1000000US' of strings in geoid
nonpop_blocks = pd.Series(nonpop_blocks).apply(lambda x: x[9:])

In [8]:
# Read in Colorado census block shapes data
# Data found at https://catalog.data.gov/dataset/tiger-line-shapefile-2019-2010-state-colorado-2010-census-block-state-based
block_shapes = gpd.read_file("data/tl_2019_08_tabblock10/tl_2019_08_tabblock10.shp")

# Filter out blocks with no population
block_shapes_pop = block_shapes[~block_shapes['GEOID10'].isin(nonpop_blocks)]

In [9]:
# Read in data containing distances to closest populated block for each well, tank, and pit, calculated using QGIS
wells_dists = gpd.read_file("data/wells_with_distance_meters/wells_with_distance_meters.shp")
tanks_dists = gpd.read_file("data/tanks_with_distance_meters/tanks_with_distance_meters.shp")
pits_dists = gpd.read_file("data/pits_with_distance_meters/pits_with_distance_meters.shp")

# Filter out sites further than 1 kilometer, or 1000 meters, from a populated block
wells_within_onekm = wells_dists[wells_dists['distance'] <= 1000]
tanks_within_onekm = tanks_dists[tanks_dists['distance'] <= 1000]
pits_within_onekm = pits_dists[pits_dists['distance'] <= 1000]

In [19]:
# Create 1 km buffers around each census tract
# To create a weighted aggregate of the oil sites in each buffered census tract
# Using CO census tract shapefile from CDPHE here: https://data- cdphe.opendata.arcgis.com/datasets/a9f5b1a67bd74b2fa22279d141625335_3/data
tracts = gpd.read_file("data/Colorado_Census_Tract_Boundaries-shp/Colorado_Census_Tract_Boundaries.shp")
tracts = tracts.drop(columns = ["OBJECTID"])

In [3]:
# Convert buffer distance from kilometers to degrees
lat_radians = 39.7392 * np.pi/180
buff_dist = 1/(111.32*np.cos(lat_radians))

# Add buffers to census tracts
tracts_buffered = tracts.apply(lambda x: x.iloc[-1].buffer(buff_dist), axis = 1) # axis = 1 to apply to each row
tracts_buffered_df = tracts
tracts_buffered_df['geometry'] = tracts_buffered

# List all instances of wells within 1km of a populated block with the corresponding buffered census tract(s) they intersect with
# Before doing spatial join of the buffered tracts and wells within one km
# Convert crs to be same
tracts_buffered_df = tracts_buffered_df.to_crs(wells_within_onekm.crs)

# Spacial join points to polygons of the buffered tracts and wells within one km
wells_joined = gpd.sjoin(tracts_buffered_df, wells_within_onekm, op = 'intersects')

# Weight each site based on CalEnviroscreen weights/distances method
# 1 : <=250m
# 0.5: <=500m
# 0.25: <=750m
# 0.1: <=1000m
# Make a column to turn into weights
wells_joined['weights'] = wells_joined['distance']

# Assign weights
wells_joined['weights'] = np.where(wells_joined['distance'] <= 250, 1, 
                                     np.where(wells_joined['distance'] <= 500, 0.5,
                                     np.where(wells_joined['distance'] <= 750, 0.25,
                                     np.where(wells_joined['distance'] <= 1000, 0.1,0))))

# Sum weights for each census tract
wells_agg = wells_joined.groupby('FIPS').sum()[['weights']].reset_index().rename(columns = {
    'FIPS':'FIPS_tract_id', 
    'weights':'wells_agg'
})

# Repeat for tanks
# List all instances of tanks within 1km of a populated block with the corresponding buffered census tract(s) they intersect with
# Using a spatial join points to polygons
tanks_joined = gpd.sjoin(tracts_buffered_df, tanks_within_onekm, op = 'intersects') 


# Make a column to turn into weights
tanks_joined['weights'] = tanks_joined['distance']

#assign weights
tanks_joined['weights'] = np.where(tanks_joined['distance'] <= 250, 1, 
                                     np.where(tanks_joined['distance'] <= 500, 0.5,
                                     np.where(tanks_joined['distance'] <= 750, 0.25,
                                     np.where(tanks_joined['distance'] <= 1000, 0.1,0))))

# Sum weights for each census tract
tanks_agg = tanks_joined.groupby('FIPS').sum()[['weights']].reset_index().rename(columns = {
    'FIPS':'FIPS_tract_id', 
    'weights':'tanks_agg'
})

# Repeat for pits
# List all instances of pits within 1km of a populated block with the corresponding buffered census tract(s) they intersect with
# Using a spatial join points to polygons
pits_joined = gpd.sjoin(tracts_buffered_df, pits_within_onekm, op = 'intersects')


# Make a column to turn into weights
pits_joined['weights'] = pits_joined['distance']

# Assign weights
pits_joined['weights'] = np.where(pits_joined['distance'] <= 250, 1, 
                                     np.where(pits_joined['distance'] <= 500, 0.5,
                                     np.where(pits_joined['distance'] <= 750, 0.25,
                                     np.where(pits_joined['distance'] <= 1000, 0.1,0))))

# Sum weights for each census tract
pits_agg = pits_joined.groupby('FIPS').sum()[['weights']].reset_index().rename(columns = {
    'FIPS':'FIPS_tract_id', 
    'weights':'pits_agg'
})

# Merge all wells, pits, and tanks weighted sums for each census tract
merged_oil = tanks_agg.merge(wells_agg, 
                             on = "FIPS_tract_id", 
                             how = 'outer').merge(pits_agg, 
                                                  on = 'FIPS_tract_id', 
                                                  how = 'outer')
oil_gas = merged_oil[['tanks_agg', 'wells_agg', 'pits_agg']].sum(axis=1)
oil_gas = oil_gas.to_frame().rename(columns = { 0 : 'oil_score'})
oil_gas["FIPS_tract_id"] = merged_oil["FIPS_tract_id"]

# Merge in census tracts with an oil score of 0
block_shapes['FIPS_tract_id'] = block_shapes['STATEFP10'] + block_shapes['COUNTYFP10'] + block_shapes['TRACTCE10']
all_tracts = block_shapes['FIPS_tract_id'].to_frame().drop_duplicates()
oil_gas = oil_gas.merge(all_tracts, on = "FIPS_tract_id", how = 'outer', validate = 'one_to_one')
oil_gas.loc[oil_gas['oil_score'].isnull(), ['oil_score']] = 0
            
# Calculate percentile rank for census tracts with a score above 0, set percentile rank to 0 if score is 0
oil_gas['oil_rank'] = oil_gas[oil_gas['oil_score'] != 0]['oil_score'].rank( 
    na_option = 'keep', 
    pct = True)*100      
oil_gas.loc[oil_gas['oil_score'] == 0, 'oil_rank'] = 0
            
# Add in Colorado state identifier
oil_gas['state'] = '08'

### Generalizing distance calculation between tract and oil/gas equipment

In [28]:
#Quick and dirty extraction of centroid of census tracts
tracts['long'] = tracts['geometry'].apply(lambda lamb: lamb.centroid.x)
tracts['lat'] = tracts['geometry'].apply(lambda lamb: lamb.centroid.y)

In [42]:
#Separating lats/lons for tracts and og equipment
tract_lat = np.array(tracts['lat'])
tract_lon = np.array(tracts['long'])
og_lat = np.array(oil_gas['lat'])
og_lon = np.array(oil_gas['long'])

In [67]:
#Multi-step process to avoid populating huge matrix with distance calculations:
#(breaking down [(lat_a - lat_b)^2 + (lon_a - lon_b)^2]^.5 )
#0. Create tiled matrices to enable matrix operations
#1. Subtract and square latitudes
#2. Subtract and square longitudes
#3. Sum the differences and take the square root

#Tiling example
x = np.array([1,2,3,4,5])
y = np.array([7,8])
print(np.tile(y, (len(x),1)))
print(np.tile(x, (len(y),1)).T)

[[7 8]
 [7 8]
 [7 8]
 [7 8]
 [7 8]]
[[1 1]
 [2 2]
 [3 3]
 [4 4]
 [5 5]]


In [68]:
#Creating tiled latitude matrices for tracts and og latitudes
tract_lat_tile = np.tile(tract_lat, (len(og_lat),1))
og_lat_tile = np.tile(og_lat, (len(tract_lat),1)).T
assert tract_lat_tile.shape == og_lat_tile.shape

#Creating tiled latitude matrices for tracts and og latitudes
tract_lon_tile = np.tile(tract_lon, (len(og_lon),1))
og_lon_tile = np.tile(og_lon, (len(tract_lon),1)).T
assert tract_lon_tile.shape == og_lon_tile.shape

In [69]:
#Subtract and square lats and lons
squared_lat_diffs = np.square(np.subtract(tract_lat_tile, og_lat_tile))
squared_lon_diffs = np.square(np.subtract(tract_lon_tile, og_lon_tile))
assert squared_lat_diffs.shape == squared_lon_diffs.shape

In [76]:
#Sum and square-root the differences
#Now we can sort / further manipulate the distance matrix to get the products we need!
dist_matrix = np.sqrt(np.add(squared_lat_diffs, squared_lon_diffs))

In [80]:
dist_matrix.shape

(84208, 1249)

In [81]:
dist_matrix

array([[2.43692983, 3.01799576, 1.34501815, ..., 1.04445702, 1.00834491,
        1.00116026],
       [2.72059593, 0.9729695 , 2.66846459, ..., 3.30223477, 3.31574877,
        3.31817327],
       [1.9220457 , 2.56593477, 0.97645704, ..., 0.54714349, 0.51005531,
        0.50256282],
       ...,
       [2.20901027, 2.59601267, 0.91649048, ..., 0.87025833, 0.8338034 ,
        0.82638846],
       [2.61579749, 3.27160938, 1.59831331, ..., 1.2275286 , 1.19320531,
        1.18644889],
       [2.60402595, 3.26801655, 1.59645974, ..., 1.21632676, 1.18211188,
        1.17538096]])